In [1]:
import piq
import datasets
import torch
from torchvision import transforms
from datasets import Dataset
from datasets import load_dataset
from transformers import pipeline
from evaluate import evaluator

In [2]:
def pil_to_pt(img):
    t = transforms.functional.pil_to_tensor(img)
    t = t.to(torch.float)
    t = t/255
    t = t.unsqueeze(0)
    return t

In [3]:
vit_b = pipeline("image-classification", model="google/vit-base-patch16-224")
task_evaluator = evaluator("image-classification")
clip_iqa = piq.CLIPIQA()
dataset = load_dataset("imagenet-1k")

In [4]:
%%time

quality = []
correct = []

for i_sample in range(dataset['validation'].num_rows):
    sample = Dataset.from_dict(dataset['validation'][i_sample:i_sample+1])
    
    img = sample[0]['image']
    x = pil_to_pt(img)
    quality.append(clip_iqa(x).numpy()[0][0])
    
    eval_results = task_evaluator.compute(
        model_or_pipeline=vit_b,
        data=sample,
        metric="accuracy",
        label_mapping=vit_b.model.config.label2id,
    )
    correct.append(eval_results['accuracy'])



KeyboardInterrupt



In [14]:
MPQv0 = Dataset.from_dict({"quality":quality,"correct":correct})
MPQv0 = MPQv0.train_test_split(0.2)
MPQv0.push_to_hub("danjacobellis/MPQv1")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]